In [1]:
import os

def train():
    import tensorflow as tf

    print("TensorFlow version: ", tf.__version__)

    mnist = tf.keras.datasets.mnist

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.summary()
    print("Training...")
    model.fit(x_train, y_train, epochs=5, validation_split=0.2)

    # Evaluate the model on test set
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test accuracy: ', score[1])

In [2]:
def fairing_run():
    import uuid
    from kubeflow import fairing
    
    CONTAINER_REGISTRY = 'kangwoo'

    namespace = 'admin'
    job_name = f'fairing-function-append-job-{uuid.uuid4().hex[:4]}'

    fairing.config.set_preprocessor("function", function_obj=train)
    fairing.config.set_builder('append', registry=CONTAINER_REGISTRY, image_name="fairing-function-append-job",
                               base_image="tensorflow/tensorflow:2.1.0-py3")

    fairing.config.set_deployer('job', namespace=namespace, job_name=job_name, cleanup=False, stream_log=True)

    fairing.config.run()

In [3]:
if __name__ == '__main__':
    fairing_run()

[I 200430 13:20:27 config:134] Using preprocessor: <kubeflow.fairing.preprocessors.function.FunctionPreProcessor object at 0x7f0c50127c88>
[I 200430 13:20:27 config:136] Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7f0c11723400>
[I 200430 13:20:27 config:138] Using deployer: <kubeflow.fairing.deployers.job.job.Job object at 0x7f0c50127d68>
[W 200430 13:20:27 append:50] Building image using Append builder...
[I 200430 13:20:27 base:107] Creating docker context: /tmp/fairing_context_l2vao2qy
[W 200430 13:20:27 base:94] /home/jovyan/.local/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
[I 200430 13:20:27 docker_creds_:234] Loading Docker credentials for repository 'tensorflow/tensorflow:2.1.0-py3'
[W 200430 13:20:30 append:54] Image successfully built in 2.9777171310051926s.
[W 200430 13:20:30 append:94] Pushing image kangwoo/fairing-function-append-job:A61842C0...
[I 200430 13:20:30 docker_creds

2020-04-30 13:20:41.733948: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-04-30 13:20:41.733993: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-04-30 13:20:41.733997: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
TensorFlow version:  2.1.0
11493376/11490434 [==============================] - 0s 0us/step
2020-04-30 13:20:43.411286: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object f

In [4]:
def fairing_fn(fn):
    import uuid
    from kubeflow import fairing

    CONTAINER_REGISTRY = 'kangwoo'

    namespace = 'admin'
    job_name = f'fairing-fn-append-job-{uuid.uuid4().hex[:4]}'


    fairing.config.set_builder('append', registry=CONTAINER_REGISTRY, image_name="fairing-fn-append-job",
                               base_image="tensorflow/tensorflow:2.1.0-py3")

    fairing.config.set_deployer('job', namespace=namespace, job_name=job_name, cleanup=False, stream_log=True)

    return fairing.config.fn(fn)

In [5]:
if __name__ == '__main__':
    remote_train = fairing_fn(train)
    remote_train()

[I 200430 13:20:54 config:134] Using preprocessor: <kubeflow.fairing.preprocessors.function.FunctionPreProcessor object at 0x7f0c50127908>
[I 200430 13:20:54 config:136] Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7f0c501279b0>
[I 200430 13:20:54 config:138] Using deployer: <kubeflow.fairing.deployers.job.job.Job object at 0x7f0c501275c0>
[W 200430 13:20:54 append:50] Building image using Append builder...
[I 200430 13:20:54 base:107] Creating docker context: /tmp/fairing_context_ivpp1a3t
[W 200430 13:20:54 base:94] /home/jovyan/.local/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
[I 200430 13:20:54 docker_creds_:234] Loading Docker credentials for repository 'tensorflow/tensorflow:2.1.0-py3'
[W 200430 13:20:56 append:54] Image successfully built in 2.5889422969994484s.
[W 200430 13:20:56 append:94] Pushing image kangwoo/fairing-fn-append-job:EAD746F0...
[I 200430 13:20:56 docker_creds_:234]

2020-04-30 13:21:08.182286: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-04-30 13:21:08.182328: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-04-30 13:21:08.182332: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
TensorFlow version:  2.1.0
11493376/11490434 [==============================] - ETA: 74
 0s 0us/step
2020-04-30 13:21:09.599674: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared